In [ ]:
# The following command can be copy-pasted to CLI but it does not really download a catalogue
# Currently it is not used
java -jar stilts.jar tapquery \
  tapurl="http://tapvizier.cds.unistra.fr/TAPVizieR/tap" \
  adql='SELECT * FROM "VIII/92/first14"' \
  ofmt=fits \
  out=./first14.fits

In [ ]:
#For step3 cooking on one source, examples

# The following codes are for single catalogue, just for reference
# # 2.  After filtering, adjust the groupIDs accordingly
# java -jar $stilts_dir tgroup in="filtered_r1" \
#     keys=GroupID \
#     aggcols="0;count;GroupCount" \
#     out=group_count.fits
# java -jar $stilts_dir tpipe in=group_count.fits \
#     cmd='select "GroupCount == 1"' \
#     out=singleton_groupids.fits

# java -jar $stilts_dir tmatch2 \
#     in1=filtered_r1.fits \
#     in2=singleton_groupids.fits \
#     matcher=exact \
#     join=all1 \
#     values1=GroupID values2=GroupID \
#     out=temp_with_gcount.fits

# java -jar $stilts_dir tpipe \
#     in=temp_with_gcount.fits \
#     cmd='addcol GroupID "!NULL_GroupID_1 ? GroupID_1 : NULL"' \
#     cmd='delcols GroupID_2' \
#     cmd='delcols GroupID_1' \
#     out=temp_with_gcount.fits
# exit

# #Note, if you don't specify the !NULL option, stilts will treat the ELSE condition (when GroupCount is NULL) as NULL, so it will give GroupSize = NULL rather than keeping the original GroupSize value.
# rpl_cmd='replacecol GroupSize "!NULL_GroupCount && GroupCount == 1 ? 1 : GroupSize"'
# # This may overlappipng with the previous command, but it is necessary to ensure that GroupSize is set to 1 for singleton groups.
# rpl_cmd2='replaceval NULL 1 "GroupSize"'
# java -jar $stilts_dir tpipe ifmt=fits \
#     in=temp_with_gcount.fits \
#     cmd="$rpl_cmd" \
#     cmd="$rpl_cmd2" \
#     out=filtered_r1_CLN.fits

# # 3. Flag the high-z and single matched sources as tier 1

# flag_cmd1='addcol source_tier "GroupSize==1 && Z_o1>=3 ? 1 : 0"'
# # flag_cmd2='replacecol source_tier "source_tier == 0 && group_z_min > 3 ? 2 : 0"'
# # flag_cmd3='replacecol source_tier "source_tier == 0 && group_z_min <= 3 && Z_o1>=3 ? 3 : 0"'
# java -jar $stilts_dir tpipe ifmt=fits in=filtered_r1_CLN.fits \
#     cmd="$flag_cmd1" \
#     out=mr1_t1.fits

# cook the grouped sources
# java -jar $stilts_dir tgroup \
#     in=filtered_r3.fits \
#     keys=GroupID \
#     aggcols='Z_o1;max;group_z_max Z_o1;min;group_z_min 0;count;GroupSize' \
#     out=group_zcheck.fits

# java -jar $stilts_dir tpipe \
#     in=group_zcheck.fits \
#     cmd='select "!NULL_GroupID && GroupSize > 1 && group_z_max > 3"' \
#     cmd='keepcols "GroupID group_z_min"' \
#     out=r3_group_hz_sources.fits

# java -jar $stilts_dir tmatch2 \
#     in1=filtered_r3.fits \
#     in2=r3_group_hz_sources.fits \
#     matcher=exact \
#     join=all1 \
#     find=all \
#     values1=GroupID values2=GroupID \
#     ocmd='delcols "GroupID GroupSize"' \
#     ocmd='addcol GroupID "!NULL_GroupID_1 ? GroupID_1 : NULL"' \
#     ocmd='addcol GroupSize "!NULL_GroupSize_1 ? GroupSize_1 : NULL"' \
#     ocmd='delcols "GroupID_1 GroupID_2 GroupSize_1"' \
#     out=filtered_r3_Tier23.fits

#         #    cmd='replacecol source_tier (
#         # source_tier < 2 && GroupID_NULL ? source_tier :
#         # (source_tier < 2 && z_min > 3 ? 3 :
#         # (source_tier < 2 ? 2 : source_tier)))' \
# exit

In [2]:
import pandas as pd
from astropy.io import fits
import numpy as np
import os
from astropy.table import Table
from astropy.coordinates import SkyCoord

In [7]:

sdss_galaxy = "../catalogues/dr16_galaxy.fits"
dr16q_short = "tmp_sdss.fits"
outname = "tmp_sdss16_comb.fits"

with fits.open(sdss_galaxy) as hdul:
    data = hdul[1].data 
    table_g = Table(data)  
df_galaxy = table_g.to_pandas()  


In [7]:
df_galaxy



,specobjid(d),SPECOBJID,PLUG_RA,PLUG_DEC,CLASS,SUBCLASS,Z,Z_ERR,RCHI2,ZWARNING,Z_PERSON,CLASS_PERSON,Z_CONF_PERSON,COMMENTS_PERSON
0,2.994897e+17,299489676975171584,146.71421,-1.041304,GALAXY,,0.021275,0.000009,1.074070,0,0.0,0,0,
1,2.994900e+17,299489951853078528,146.91945,-0.990492,GALAXY,,0.213925,0.000052,1.331336,0,0.0,0,0,
2,2.994905e+17,299490501608892416,146.85983,-0.808902,GALAXY,,0.126554,0.000062,1.338671,0,0.0,0,0,
3,2.994908e+17,299490776486799360,146.76339,-0.810433,GALAXY,,0.448645,0.000168,1.435746,0,0.0,0,0,
4,2.994911e+17,299491051364706304,146.96390,-0.545003,GALAXY,STARBURST,0.055988,0.000011,1.315894,0,0.0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863630,1.084494e+19,10844941542625660928,141.89192,28.041277,GALAXY,,1.037471,0.000178,1.032615,0,0.0,0,0,
2863631,1.084494e+19,10844941817503567872,141.83234,28.014509,GALAXY,,0.742518,0.000056,1.076094,0,0.0,0,0,
2863632,1.084494e+19,10844942367259381760,141.92392,28.090031,GALAXY,,1.056773,0.000149,1.094800,0,0.0,0,0,
2863633,1.084494e+19,10844942642137288704,141.97961,27.957674,GALAXY,,1.012589,0.000177,1.138485,0,0.0,0,0,


In [10]:
source_coords = SkyCoord(ra=df_galaxy['PLUG_RA'][:500], dec=df_galaxy['PLUG_DEC'][:500], unit='deg')
# make a sdss style source name based on RA and DEC
ra = source_coords.ra.to_string(unit='hour', sep=':', precision=2, pad=True)
dec = source_coords.dec.to_string(unit='deg', sep=':', precision=1, pad=True)
for i in range(len(dec)):
    print(dec[i])
    if dec[i][0] == '-':
        dec[i] = dec[i]
    else:
        dec[i] = '+' + dec[i]  # add a plus sign for positive declinations

sdss_name = np.char.add('g', np.char.add(np.char.replace(ra, ':', ''), np.char.replace(dec, ':', '')))
sdss_name

-01:02:28.7
-00:59:25.8
-00:48:32.0
-00:48:37.6
-00:32:42.0
-01:14:26.7
-00:33:24.8
-00:39:07.9
-00:45:36.9
-00:40:13.5
-00:35:31.9
-00:59:17.8
-00:45:54.5
-00:57:05.0
-00:39:46.3
-00:45:33.7
-00:06:18.5
-00:09:36.3
-00:04:28.4
-00:20:32.3
-00:18:23.3
-00:21:14.6
00:01:44.0
-00:02:08.6
-00:29:37.7
-00:09:54.8
-00:17:33.4
-00:15:59.4
-01:05:05.1
-01:03:39.0
-01:07:50.0
-00:59:14.6
-00:57:26.0
-01:10:51.0
-00:50:44.8
-00:49:00.0
-00:48:47.6
-00:36:41.6
-00:40:05.7
-00:42:48.2
-00:18:49.2
-00:27:50.7
-00:18:38.4
-00:14:31.2
-00:30:45.6
-00:24:47.4
-00:29:40.9
-00:13:07.6
-00:16:26.5
-00:11:24.2
-00:10:05.6
-00:11:10.9
-00:29:12.5
-00:05:56.2
00:00:57.5
00:01:35.0
-01:09:10.0
-01:09:28.9
-01:07:09.2
-00:45:45.4
-00:41:18.2
-01:00:29.4
-00:58:30.8
-00:47:35.1
-00:52:05.3
-00:40:51.5
-00:40:55.9
-01:11:23.2
-00:56:26.9
-00:44:29.9
-00:26:03.6
-00:31:28.9
-00:11:29.4
-00:24:30.2
-00:27:56.5
-00:22:09.5
-00:23:11.5
-00:15:07.9
-00:17:28.6
-00:11:32.2
00:01:37.1
-00:19:52.8
-00:06:12.1
-00:16:0

array(['g094651.41-010228.7', 'g094740.67-005925.8',
       'g094726.36-004832.0', 'g094703.21-004837.6',
       'g094751.34-003242.0', 'g094625.73-011426.7',
       'g094654.71-003324.8', 'g094658.59-003907.9',
       'g094622.25-004536.9', 'g094744.18-004013.5',
       'g094748.00-003531.9', 'g094631.60-005917.8',
       'g094630.86-004554.5', 'g094629.48-005705.0',
       'g094725.85-003946.3', 'g094751.24-004533.7',
       'g094741.77-000618.5', 'g094805.61-000936.3',
       'g094738.59-000428.4', 'g094749.46-002032.3',
       'g094740.82-001823.3', 'g094842.33-002114.6',
       'g094919.08+000144.0', 'g094859.53-000208.6',
       'g094844.83-002937.7', 'g094827.51-000954.8',
       'g094854.17-001733.4', 'g094821.53-001559.4',
       'g094615.75-010505.1', 'g094617.46-010339.0',
       'g094513.32-010750.0', 'g094543.87-005914.6',
       'g094617.12-005726.0', 'g094517.64-011051.0',
       'g094603.08-005044.8', 'g094526.48-004900.0',
       'g094514.83-004847.6', 'g094506.81-0036

In [6]:
sdss_galaxy_sup = "../catalogues/dr16_sup_galaxy.fits"
with fits.open(sdss_galaxy_sup) as hdul:
    data = hdul[1].data 
    table_g2 = Table(data)  
df_galaxy_sup = table_g2.to_pandas()

In [37]:
# df_galaxy_sup.replace([-32768,-2147483648], 'NA', inplace=True)
# #统计该列里等于0的列的数目
print(len(df_galaxy_sup))
print(df_galaxy_sup['f_zsp'].value_counts().get(0, 0))
df_update = df_galaxy_sup[df_galaxy_sup['f_zsp'] != 0]
df_update = df_update.reset_index(drop=True)

print(df_update.columns)
df_update
# 统计里面zph = NaN 的数量
print(df_update['zph'].isna().sum())
print(df_update['zsp'])

328440
1796
Index(['objID', 'RA_ICRS', 'DE_ICRS', 'class', 'clean', 'e_RA_ICRS',
       'e_DE_ICRS', 'umag', 'gmag', 'rmag', 'imag', 'zmag', 'e_umag', 'e_gmag',
       'e_rmag', 'e_imag', 'e_zmag', 'zsp', 'e_zsp', 'f_zsp', 'spCl', 'subCl',
       'pmRA', 'pmDE', 'e_pmRA', 'e_pmDE', 'sigRA', 'sigDE', 'zph', 'e_zph',
       '<zph>', 'Q', 'SDSS16', 'Sp-ID'],
      dtype='object')
227939
0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
326639   NaN
326640   NaN
326641   NaN
326642   NaN
326643   NaN
Name: zsp, Length: 326644, dtype: float32


In [39]:
# df_update=df_update.replace([-32768,-2147483648], np.nan, inplace=True)
# df_update['sdss_name'] = pd.Series(dtype='str')
# df_update['Z'] = pd.Series(dtype='float')
# df_update['SOURCE_Z'] = pd.Series(dtype='str')
# for i in range(len(df_update)):
#     df_update.loc[i, 'sdss_name'] = df_update['SDSS16'][i].replace('SDSS J', 'g')
#     if not pd.isna(df_update['zsp'][i]):  # 使用 pd.isna() 而非对象方法
#         df_update.loc[i, 'Z'] = df_update['zsp'][i]
#         df_update.loc[i, 'SOURCE_Z'] = np.nan
#     else:
#         if not pd.isna(df_update['zph'][i]):
#             df_update.loc[i, 'Z'] = df_update['zph'][i]
#             df_update.loc[i, 'SOURCE_Z'] = 'phot'
#         else:
#             df_update.loc[i, 'Z'] = np.nan
#             df_update.loc[i, 'SOURCE_Z'] = 'NOT_CLEAN'

df_update['sdss_name'] = df_update['SDSS16'].str.replace('SDSS J', 'g')

# 初始化所有行为 NaN
df_update['Z'] = np.nan
df_update['SOURCE_Z'] = np.nan

# 有光谱红移的情况
mask_zsp = ~pd.isna(df_update['zsp'])
df_update.loc[mask_zsp, 'Z'] = df_update.loc[mask_zsp, 'zsp']
# SOURCE_Z 保持为 NaN

# 无光谱红移但有光度红移的情况
mask_zph = (~mask_zsp) & (~pd.isna(df_update['zph']))
df_update.loc[mask_zph, 'Z'] = df_update.loc[mask_zph, 'zph']
df_update.loc[mask_zph, 'SOURCE_Z'] = 'phot'

# 两种红移都没有的情况
mask_none = (~mask_zsp) & (~mask_zph)
df_update.loc[mask_none, 'SOURCE_Z'] = 'NOT_CLEAN'

/var/folders/71/gshyfsc16lv27mjb4j1d21000000gn/T/ipykernel_9088/4274680342.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'phot' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_update.loc[mask_zph, 'SOURCE_Z'] = 'phot'


In [40]:
df_update

,objID,RA_ICRS,DE_ICRS,class,clean,e_RA_ICRS,e_DE_ICRS,umag,gmag,rmag,...,sigDE,zph,e_zph,<zph>,Q,SDSS16,Sp-ID,sdss_name,Z,SOURCE_Z
0,1237646011620131287,0.205084,0.221903,3,False,0.213,0.266,23.027000,22.667000,21.420000,...,NaN,NaN,NaN,NaN,2,SDSS J000049.22+001318.8,,g000049.22+001318.8,NaN,NOT_CLEAN
1,1237663543696097772,0.220545,0.223351,3,False,0.132,0.187,26.441000,22.848000,26.023001,...,NaN,NaN,NaN,NaN,3,SDSS J000052.93+001324.0,,g000052.93+001324.0,NaN,NOT_CLEAN
2,1237666648958763365,0.238102,0.198409,3,False,0.079,0.066,23.457001,21.309000,20.080000,...,285.059998,NaN,NaN,NaN,3,SDSS J000057.14+001154.2,,g000057.14+001154.2,NaN,NOT_CLEAN
3,1237667172926227075,0.238119,0.198445,3,False,0.042,0.032,25.937000,21.243999,19.987000,...,237.419998,NaN,NaN,NaN,3,SDSS J000057.14+001154.4,,g000057.14+001154.4,NaN,NOT_CLEAN
4,1237657363222495501,0.215874,0.224069,3,False,0.173,0.152,22.278000,22.073000,22.878000,...,NaN,NaN,NaN,NaN,2,SDSS J000051.80+001326.6,,g000051.80+001326.6,NaN,NOT_CLEAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326639,1237666308021944805,359.766066,24.038759,3,True,0.023,0.028,24.808001,21.941999,20.014999,...,0.000000,0.3918,0.0287,0.3809,3,SDSS J235903.85+240219.5,,g235903.85+240219.5,0.3918,phot
326640,1237666308021944804,359.766066,24.038759,3,False,0.023,0.028,24.797001,21.923000,19.993000,...,NaN,NaN,NaN,NaN,3,SDSS J235903.85+240219.5,,g235903.85+240219.5,NaN,NOT_CLEAN
326641,1237666308021879410,359.727385,24.021728,3,True,0.293,0.191,23.209999,22.271999,21.136000,...,NaN,0.4186,0.1062,0.4159,3,SDSS J235854.57+240118.2,,g235854.57+240118.2,0.4186,phot
326642,1237666308021879614,359.704320,24.054441,3,True,0.189,0.231,22.601999,22.802000,21.371000,...,NaN,0.4985,0.0911,0.5204,3,SDSS J235849.03+240315.9,,g235849.03+240315.9,0.4985,phot


In [43]:


new_df_galaxy_2 = pd.DataFrame(
    {
        'SDSS_NAME': df_update['sdss_name'],
        'RA': df_update['RA_ICRS'],
        'DEC': df_update['DE_ICRS'],
        'CLASS': 'GALAXY',
        'Z': df_update['Z'],
        'ZWARNING': df_update['f_zsp'],
        'SOURCE_Z': df_update['SOURCE_Z']
    }
)
new_df_galaxy_2['ZWARNING'] 

0        -32768
1        -32768
2        -32768
3        -32768
4        -32768
          ...  
326639   -32768
326640   -32768
326641   -32768
326642   -32768
326643   -32768
Name: ZWARNING, Length: 326644, dtype: int16